## Setup

In [11]:
from eesr import interface
from pandas import Timestamp, DataFrame
import pickle
import os
import warnings
import timeit
warnings.filterwarnings("ignore")

In [12]:
# Globals
dc_trace = "Input\opendc_out.csv"
key_path = "Input\entsoe_token.txt"
results_pre_path = "Results"

In [14]:
def run_experiment(name, start: Timestamp, country="NL", tz="Europe/Amsterdam", caching=True, green_ratio=None, subname=None, make_out=True):
    if subname is None:
        path = os.path.join(results_pre_path, name)
    else:
        path = os.path.join(results_pre_path, name, subname)
    exists = os.path.exists(path)
    if not exists:
        os.makedirs(path)
    
    def store_df(df: DataFrame):
        pickle_path = os.path.join(path, "df.pkl")
        df.to_pickle(pickle_path)

    results = os.path.join(path, "results.json")
    df = interface.opendc_grid_analysis(dc_trace, key_path, start, country, results, tz, caching, green_ratio)

    if make_out:
        store_df(df)

        report_path = os.path.join(path, "report.html")
        report = interface.generate_compact_profile(results, "sus_prof", report_path, True)
        interface.to_image(report)

# Main Experiments

## 1 Caching Performance

#### Timing no cache

In [4]:
%%timeit -n 1 -r 4
start = Timestamp(year=2020, month=9, day=4, hour=15, tz="Europe/Amsterdam")
run_experiment("caching", start, caching=False, make_out=False)

Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
2min 22s ± 2.93 s per loop (mean ± std. dev. of 4 runs, 1 loop each)


#### Timing cache

In [17]:
%%timeit -n 1 -r 4
start = Timestamp(year=2020, month=9, day=4, hour=15, tz="Europe/Amsterdam")
run_experiment("caching", start, caching=True, make_out=False)

Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
2.64 s ± 46.3 ms per loop (mean ± std. dev. of 4 runs, 1 loop each)


#### Bandwidth savings (with wireshark)

In [18]:
start = Timestamp(year=2020, month=9, day=4, hour=15, tz="Europe/Amsterdam")
run_experiment("caching", start, caching=False, make_out=False)

Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch


## 2 Baseline: Assume Proportional to Grid Production

In [5]:
start = Timestamp(year=2020, month=9, day=4, hour=15, tz="Europe/Amsterdam")
run_experiment("baseline", start)

Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'


## 3 Optimistic: Assume 88% Green

In [6]:
start = Timestamp(year=2020, month=9, day=4, hour=15, tz="Europe/Amsterdam")
run_experiment("optimistic", start, green_ratio=0.88)

Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'


## 4 Seasonal Comparison

In [7]:
for i in range(0, 12, 3):
    month = 1 + i
    start = Timestamp(year=2020, month=month, day=1, hour=15, tz="Europe/Amsterdam")
    run_experiment("season", start, subname=str(month))

Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElement

## 5 Yearly Comparison

In [8]:
for i in range(5):
    year = 2016 + i
    start = Timestamp(year=year, month=9, day=4, hour=15, tz="Europe/Amsterdam")
    run_experiment("yearly", start, subname=str(year))

Only one level for prod fetch
Only one level for prod fetch
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
Only one level for prod fetch
Only one level for prod fetch
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementB

## 6 Different Countries

In [9]:
countries = {
    "FR" : "Europe/Paris",
    "GB" : "Europe/London",
    "SE_3" : "Europe/Stockholm",
    "BE" : "Europe/Brussels",
    "DE_LU" : "Europe/Berlin"
}

for country in countries.keys():
    start = Timestamp(year=2020, month=9, day=4, hour=15, tz=countries[country])
    run_experiment("countries", start, country, countries[country], subname=country)

Only one level for prod fetch
Only one level for prod fetch
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
Only one level for prod fetch
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' h